In [88]:
from modules import abrindo_dados, limpeza_de_dados, reclassificando_vetores, adicionando_dados, caixa_de_ferramentas_vetores
import os
import geopandas as gpd
import pandas as pd

In [89]:
#Abrindo arquivos
vault_files = "/home/luisthethormes/01_sol/geo-dev/Sol-luis.github.io/data/"

limite_produtor = abrindo_dados.process_files(vault_files, "pol_props*", "geojson", zipped = False)
limite_produtor = limite_produtor[['car','produtor', 'municipio', 'des_condic', 'area_hectares', 'geometry']]
desmates = abrindo_dados.process_files(vault_files, "prodes_deforestation_*", "geojson", zipped = False)
desmates = desmates[['year', 'image_date', 'geometry']]
es_uso_solo_vetor = abrindo_dados.process_files(vault_files, "es_uso_solo_vetor_reclassificado_pt*", "geojson", zipped = False)

Dimensões do(s) arquivo(s) ['pol_props_ES.geojson']: (74, 15)
Dimensões do(s) arquivo(s) ['prodes_deforestation_clip.geojson']: (60, 16)
Dimensões do(s) arquivo(s) ['es_uso_solo_vetor_reclassificado_pt.geojson']: (1761, 3)


In [90]:
### Calculando área em hectares ###
desmates = adicionando_dados.add_area(desmates)
es_uso_solo_vetor = adicionando_dados.add_area(es_uso_solo_vetor)

In [91]:
#Unindo espacialmente
limite_produtor_left_desmates = caixa_de_ferramentas_vetores.spatial_joining(limite_produtor, desmates, how='left', predicate='intersects')
limite_produtor_left_uso_solo = caixa_de_ferramentas_vetores.spatial_joining(limite_produtor, es_uso_solo_vetor, how='left', predicate='intersects')

crs are the same
crs are the same


In [92]:
limite_produtor_left_desmates[['area_hectares_right', 'year']] =  limite_produtor_left_desmates[['area_hectares_right', 'year']].fillna(0)


In [93]:
print(limite_produtor_left_desmates.year.value_counts())

0.0       37
2004.0    30
2008.0    11
2006.0     9
2019.0     6
2014.0     2
2021.0     2
2016.0     1
2018.0     1
2010.0     1
2013.0     1
2011.0     1
2023.0     1
Name: year, dtype: int64


In [94]:
limite_produtor_left_desmates_2015 = limite_produtor_left_desmates[limite_produtor_left_desmates['year'] == 2015]
limite_produtor_left_desmates_2015

,car,produtor,municipio,des_condic,area_hectares_left,geometry,index_right,year,image_date,area_hectares_right


In [95]:
unique_combinations = limite_produtor_left_desmates[['car', 'produtor', 'area_hectares_left']].drop_duplicates()
print(unique_combinations.shape[0]) 

73


In [96]:
#pivoting results by year and car while suming area_hectares
pivoted_deforestation = pd.pivot_table(limite_produtor_left_desmates,
                                        index=['car','produtor', 'municipio','area_hectares_left'],
                                          columns='year',
                                            values='area_hectares_right',
                                              aggfunc='sum',
                                                dropna = True,
                                                  fill_value=0)
pivoted_deforestation_r = pivoted_deforestation.reset_index()
pivoted_deforestation_r

year,car,produtor,municipio,area_hectares_left,0.0,2004.0,2006.0,2008.0,2010.0,2011.0,2013.0,2014.0,2016.0,2018.0,2019.0,2021.0,2023.0
0,ES-3200102-0DA7657B39EE4D9E9906D67D98CE1BAF,Jose,Afonso Claudio,26.8366,0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.7631,0.0,0.0,0.0
1,ES-3200102-33A4087681AD4078A3F86BEF1D48D241,Luciano,Afonso Claudio,68.2777,0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,4.9069,0.0000,0.0,0.0,0.0
2,ES-3200102-5E99A1B806DF40C59187CBF661569204,Almir,Afonso Claudio,30.0864,0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
3,ES-3200102-664BB749255B45B2B4ED8802E2D13EBF,Almir,Afonso Claudio,22.4866,0,2.7257,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
4,ES-3200102-891D619317D84CCEAD2163DB4FB056E0,Almir,Afonso Claudio,26.0686,0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,ES-3204906-8E7FD04201B440E6907B58E6AD877F11,Rodnei,Sao Mateus,29.5403,0,1.3777,0.0000,0.0143,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
69,ES-3204955-639E160E3A27447DA5FD710114A9A034,Erivelton,Sao Roque do Canaa,21.7835,0,1.0763,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
70,ES-3204955-BE1651DD25E7489A8755D5DB16FC4037,Wellington,Sao Roque do Canaa,33.6133,0,0.5138,2.2168,1.4136,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0
71,ES-3205176-314D53EDA025456AAB8608B3518406F8,Gustavo,Vila Valerio,10.0746,0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0


In [97]:
limite_produtor_left_uso_solo = pd.pivot_table(limite_produtor_left_uso_solo,
                                        index=['car','produtor', 'municipio','area_hectares_left'],
                                          columns='classe_uso_solo_mapbiomas',
                                            values='area_hectares_right',
                                              aggfunc='sum',
                                                  fill_value=0)                                                
limite_produtor_left_uso_solo_r = limite_produtor_left_uso_solo.reset_index()
limite_produtor_left_uso_solo_r

classe_uso_solo_mapbiomas,car,produtor,municipio,area_hectares_left,Afloramento rochoso,Café,Corpo de água,Floresta alagável,Formação florestal,Lavouras perenes,Lavouras temporárias,Outros usos agrícolas,Pastagem,Silvicultura
0,ES-3200102-0DA7657B39EE4D9E9906D67D98CE1BAF,Jose,Afonso Claudio,26.8366,0.0,5.6785,0.0000,0.0,9.0341,0.0000,0.0,8.1738,1.5487,2.5811
1,ES-3200102-33A4087681AD4078A3F86BEF1D48D241,Luciano,Afonso Claudio,68.2777,0.0,4.4682,0.0000,0.0,23.0286,0.0000,0.0,28.5274,11.7718,0.0000
2,ES-3200102-5E99A1B806DF40C59187CBF661569204,Almir,Afonso Claudio,30.0864,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0,11.2680,18.7518,0.0000
3,ES-3200102-664BB749255B45B2B4ED8802E2D13EBF,Almir,Afonso Claudio,22.4866,0.0,2.5792,0.0000,0.0,6.1900,0.5159,0.0,17.3667,7.9093,0.0000
4,ES-3200102-891D619317D84CCEAD2163DB4FB056E0,Almir,Afonso Claudio,26.0686,0.0,12.5515,0.0000,0.0,5.5021,0.0000,0.0,4.9004,3.0090,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,ES-3204906-8E7FD04201B440E6907B58E6AD877F11,Rodnei,Sao Mateus,29.5403,0.0,7.3028,1.2170,0.0,2.4343,0.0000,0.0,14.4318,3.8252,0.0000
69,ES-3204955-639E160E3A27447DA5FD710114A9A034,Erivelton,Sao Roque do Canaa,21.7835,0.0,5.0897,0.0000,0.0,7.5913,0.0000,0.0,8.1092,0.6039,0.0000
70,ES-3204955-BE1651DD25E7489A8755D5DB16FC4037,Wellington,Sao Roque do Canaa,33.6133,0.0,0.0000,0.5179,0.0,2.5897,0.0000,0.0,23.7391,6.7334,0.0000
71,ES-3205176-314D53EDA025456AAB8608B3518406F8,Gustavo,Vila Valerio,10.0746,0.0,0.0000,0.7816,0.0,1.8238,0.0000,0.0,22.2329,0.4342,0.0000
